# Interacting with Campaigns and Filters <a class="anchor" id="top"></a>

Now that Unicorn Post has trained models for 2 different use cases (User Personalization and Personalized Reranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters you created earlier in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`News_01_Data_Layer.ipynb`](News_01_Data_Layer.ipynb), and [`News_02_Training_Layer.ipynb`](News_02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have two deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with the Campaigns and get recommendations. 

We will interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building in this notebook highlighted in blue with a dashed outline.

![Workflow](Images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [176]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

pd.set_option('max_colwidth', 3000) # allows us to see more text for our news articles

In [152]:
#retrieves previously stored variables 
%store -r 

In [99]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with Campaigns <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the models have been trained, lets have a look at the recommendations we can get for our users!

### User Personalization Model

"User Personalization " requires a user as input, and it will return the items it thinks the customer is most likely to interact with next.

The cells below will handle getting recommendations from the "User Personalization Model" and rendering the results. Let's see what the recommendations are for a user.

We will be using the `campaignArn`, the `userId`, as well as the number or results we want, `numResults`.

### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

#### Sample User ID's
 -8845298781299428018
 -1032019229384696495
 -1130272294246983140
 344280948527967603
 -445337111692715325

In [100]:
sample_user = str(-8845298781299428018)

In [118]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    context={
        'user_device_type': 'UnknownAgent'
    },
    metadataColumns = {"ITEMS": ["training_text"]},
    numResults = 5
)

In [119]:
print(get_recommendations_response)

{'ResponseMetadata': {'RequestId': 'cceeec29-1f3a-4751-8268-b6771960453d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 05 Jul 2024 20:24:08 GMT', 'content-type': 'application/json', 'content-length': '7172', 'connection': 'keep-alive', 'x-amzn-requestid': 'cceeec29-1f3a-4751-8268-b6771960453d'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '3566197569262766169', 'score': 0.9761124, 'metadata': {'training_text': 'Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forwarding to external addresses

A little hard to read - lets make a dataframe

In [120]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [121]:
recommendations_df

,itemId,score,metadata
0,3566197569262766169,0.976112,"{'training_text': 'Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forwarding to external addresses, configure sendas aliases through external providers, use HTML in vacation messages, and manipulate settings for gmail.com accounts. More settings features like mailbox delegate support will be added over time. Most settings endpoints work for any Google Apps or Gmail account, but sensitive operations like modifying aliases or forwarding are restricted to service accounts with domainwide authority. The existing Email Settings API in the Admin SDK is deprecated and will be turned down on July 7, 2017 in favor of the updated Gmail API. Google has provided a migration guide to help clients transition.'}"
1,8482750322470893687,0.001896,"{'training_text': 'Google consolidates Hangouts On Air into YouTube Live Google is retiring Hangouts On Air and moving livestreaming features to YouTube Live to consolidate products. Google announced that Google+ Hangouts On Air will no longer be available starting September 12. Users will be asked to use YouTube Live for livestreaming instead. Google first launched Hangouts On Air in 2011 for select users and opened it up to everyone in 2012. In 2013, Google launched YouTube Live and has now decided to consolidate the two products. After September 12, users won't be able to schedule new Hangouts On Air events. Existing events after this date will need to be moved to YouTube Live. Hangouts On Air will still exist in some form, just not tied to Google+. Users will need to use YouTube Live's builtin scheduling feature instead. To use Hangouts On Air with YouTube Live: Go to Live Streaming Events in Creator Studio, click New live event, select Quick (using Google Hangouts On Air), title the event, schedule it or go live now, and broadcast with Hangouts. Recorded events will still be on YouTube. The only Google+ link will be to see event content in readonly Activity Log. Google began dropping the Google+ requirement across products in 2015, a process still ongoing. Last week Google Play dropped the requirement for reviews. Next month, Hangouts On Air scheduling will move off Google+.'}"
2,2573252627510191315,0.001697,"{'training_text': 'Setting autoexpiry dates for Google Drive file sharing links A guide explains how to create temporary access links for Google Drive files that automatically expire after a chosen time period, using a thirdparty web app. The article explains how to set autoexpiry dates for shared Google Drive links. This allows you to share files that are only accessible for a limited time. While Google Drive allows setting expiration dates, this is only available for paid Google Workspace accounts. The article provides instructions for free Google account users to create temporary links that autoexpire after a chosen time period. The steps are: 1. Go to labnol.org/expire and authorize the web app to access your Google Drive. 2. Open the File Picker and select the file or folder to share. 3. Enter the email addresses of users to share with. Specify if they get viewer or editor access. 4. Set the time period after which access will be revoked. 5. Click \""Set Expiration\"" and the access will be automatically removed after the set date/time. The article also mentions the Google Drive Auditor addon to analyze permissions on shared files. And the autoexpiry app lists files set to expire.'}"
3,1964631817676172382,0.001558,"{'training_text': 'Google relea

What has this user viewed previously

In [122]:
viewed_interactions = interaction_data[interaction_data['user_id'].astype(str) == sample_user].sort_values('timestamp', ascending=False)[0:11]

In [123]:
viewed_interactions

,timestamp,event_type,item_id,user_id,session_id,user_device_type
69083,1487360168,FOLLOW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69085,1487360167,COMMENT CREATED,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69082,1487359849,VIEW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69722,1487069095,VIEW,-8900113512825364282,-8845298781299428018,8663979798581613597,NonMobile
68453,1485973216,VIEW,-532999578436827210,-8845298781299428018,-5430065457414428568,NonMobile
68450,1485973198,VIEW,-532999578436827210,-8845298781299428018,-5430065457414428568,NonMobile
66947,1485900025,FOLLOW,-532999578436827210,-8845298781299428018,-4872963759822924080,NonMobile
66946,1485900025,COMMENT CREATED,-532999578436827210,-8845298781299428018,-4872963759822924080,NonMobile
66945,1485899837,VIEW,-532999578436827210,-8845298781299428018,-4872963759822924080,NonMobile
66803,1484848565,VIEW,7419040071212162906,-8845298781299428018,-6140011183054037117,NonMobile


Lets take a closer look at the most recent articles interacted with by this user

In [137]:
most_recent_five_articles = viewed_interactions.item_id.unique().astype(str).tolist()

In [138]:
most_recent_five_articles_metadata = articles_mlfeatures[articles_mlfeatures['item_id'].isin(most_recent_five_articles)]

In [139]:
most_recent_five_articles_metadata[['item_id', 'training_text']]

,item_id,training_text
2869,8526042588044002101,"Cloud Native enables efficient, automated infrastructure through modular architecture Cloud Native is an approach to build modular, automated systems that can manage complexity and accelerate development. It utilizes practices like microservices, infrastructure as code, and CI/CD. Cloud Native is a way of structuring teams, culture, and technology to utilize automation and architectures to manage complexity and unlock velocity. The real value of Cloud Native goes beyond just the technologies associated with it. Cloud Native enables more efficient and happier teams by breaking large problems into smaller pieces and reducing manual work through automation. It leads to more reliable infrastructure and applications by building in automation to handle failures. Cloud Native also enables auditable, visible, and debuggable complex applications through the tools used to build them. It allows developers to create more secure applications through least trust principles. Finally, Cloud Native opens up algorithmic automation for more efficient use of resources like intelligent scheduling. We are still early in applying Cloud Native outside of technology companies, but it brings many benefits. Heptio is excited to help bring Cloud Native's advantages to the wider IT industry."
2982,7419040071212162906,"Google search traffic hack by targeting \""best of year\"" keywords Adding the current year to keywords can drive search traffic by targeting fresh annual results. But you need a monetization strategy like affiliate links. The article discusses how adding \""2016\"" or \""2017\"" to Google search queries can send huge amounts of traffic to websites. The author first got the idea when searching for \""best business books 2016\"" and realized many people search with the current year to get the most recent results. He tested this on his own site by publishing an article with \""2016\"" in the headline and got tens of thousands of visitors. Another example is the site 10beasts.com which was launched in 2016 with articles about the \""best\"" products that year. Just 4 months after launch the site was getting 100k+ visitors per month and earning $4,687 in commissions. The site BestProducts.com also dominated Google in 2016 for year based terms, getting millions of visitors. To find profitable keywords, the author suggests using Google Trends, Google Keyword Planner, and reverse engineering sites like Amazon. Overall the tactic can drive huge organic search traffic if you target the right keywords, though you need a monetization strategy like affiliate marketing to profit. The key is finding \""long tail\"" keywords with buyer intent that are less competitive. With some effort, the strategy provides a big SEO opportunity in 2017."
3040,-532999578436827210,"IBM launches cloud graph database service using open source technologies IBM Graph provides developers an interactive graph database to learn Gremlin query language. It aims to make graph technology more accessible. IBM has launched a new cloud based graph database service called IBM Graph, built on open source Titan and Apache TinkerPop technologies. The service is designed to make graph databases more accessible to developers by providing an interactive tutorial playground to help them learn Gremlin query language. The goal is to get developers up and running with graph technology quickly. IBM Graph provides up to 500MB of free storage and 25,000 free API calls per month. Additional storage and API calls are charged per GB and per 1000 calls. The service runs on IBM data centers on Cassandra for storage, Elasticsearch for indexing, and the open source Titan and TinkerPop technologies. IBM chose not to modify or fork these open source projects in order to maintain compatibility. IBM Graph is focused on transactional workloads by providing a secondary operational data store for real time graph queries from other production applications. Analytic w

We see that the user has previously red information on cloud applications and google in particular. In fact one of our recommendations is actually in the users recent interaction history. This is not ideal lets use the fitler we created earlier to exclude this data. We will put in a timestamp cutoff of 0 for now to include all articles. And we will also include all availible genres in our filter for the moment.

In [140]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5,
    filterArn = genre_filter_arn,
    filterValues = {"CUTOFF": "\"0\"", "GENRELIST": "\"tech\",\"non tech\",\"cloud provider news\",\"crypto currency\""},
    metadataColumns = {"ITEMS": ["training_text"]}
)

In [141]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [142]:
recommendations_df

,itemId,score,metadata
0,-6623581327558800021,0.124971,"{'training_text': 'Google's Spanner database offers global replication and consistency Google is now offering its globally distributed Spanner database as a cloud service. The unique time synchronization could benefit global businesses. About a decade ago, Google engineers built a global database called Spanner that works across multiple data centers around the world. Spanner behaves as if it's in one place, allowing Google to replicate data and make changes globally without conflicts. This provides consistency across Google services like Gmail and AdWords. Spanner achieves this by using GPS receivers, atomic clocks, and time synchronization technology called TrueTime to establish a common time reference across data centers. This allows for coordination between distributed systems faster than the speed of light. TrueTime is unique to Google. Spanner gives Google a competitive advantage in cloud computing against rivals like Amazon and Microsoft. Google hopes Spanner's global replication and consistency will appeal to global businesses and financial institutions. A supply chain company called JDA is already testing Spanner. Spanner was previously only useful to Google, but now Google is offering it as a cloud service so other companies can benefit from its global scale and unique time synchronization abilities. This could allow businesses to more easily expand globally and replicate data across regions.'}"
1,717471739656838182,0.067480,"{'training_text': 'Evernote migrates from own data centers to Google Cloud Evernote moved its popular note taking service to Google Cloud for machine learning capabilities, flexibility, and to avoid running its own infrastructure. Evernote decided last year to move away from running its own data centers and start using the public cloud to operate its popular notetaking service. It signed up to work with Google Cloud Platform. Google sent engineers to help Evernote with the migration. Evernote wanted to leverage the cloud for new machine learning features and flexibility. The migration required adapting Evernote's backend, which was built assuming onpremises data centers. Google's expertise in unstructured data and machine learning made it a good fit over AWS or Azure. The migration had to be done without losing any of Evernote's 3 petabytes of data or disrupting the live service. Evernote reworked parts of its application like networking and storage to work on Google's cloud. Google engineers provided 24/7 support and even came onsite to help. The main migration took around 10 days. The cloud removed Evernote's need to maintain its own data centers and keep up with infrastructure advances. It provides reduced latency for international users through Google's network. Evernote is still migrating some final user attachments but all data is now encrypted at rest. The company has more freedom to leverage cloud services for new features.'}"
2,-1415040208471067980,0.039467,"{'training_text': 'Google Cloud Platform releases icon library for diagrams Google Cloud Platform published downloadable icons and sample diagrams to help visually represent GCP infrastructure and services. Google Cloud Platform has released downloadable icon sets and sample diagrams to help developers, architects, and partners visually represent complex cloud infrastructure in documents and presentations. The icons cover GCP products, services, and elements that can be combined with other providers' icons to show hybrid/multicloud setups. Over 50 sample diagrams are available in Slide and PowerPoint formats to help kickstart new diagrams. As more products launch, Google will add icons so users should revisit the icon library. This allows accurate, uptodate visual representations of Google Cloud Platform architectures and services.'}"
3,8482750322470893687,0.038279,"{'training_text': 'Google consolidates Hangouts On Air into YouTube Live Google is retiring Hangouts On Air and m

Much better a variety of articles about google technical products, similar to but different than the users recent reading history - which is great. Now lets send in some interactions with a different sort of article.

Below are five sample to choose from: 

-4996336942690402156 -1878128207048892154 -3058031327323357308 -1633984990770981161 -1633984990770981161

In [ ]:
most_recent_five_articles_metadata[most_recent_five_articles_metadata.ite['item_id', 'training_text']]

In [149]:
articles_mlfeatures[articles_mlfeatures.item_id.isin(['-4996336942690402156', '-1878128207048892154', '-3058031327323357308', '-1633984990770981161', '90383487344892230'])]

,creation_timestamp,item_id,lang,article_genre,training_text
2095,1471520397,-1633984990770981161,pt,tech,"UX vs UI design interconnected roles optimizing user experience An article explains the difference between UX and UI design. It argues they are interconnected roles focused on understanding and optimizing user needs and emotions. The article discusses the difference between UX (user experience) and UI (user interface) design. It provides some historical context, explaining that 15 years ago there were not specialized roles like UX and UI designer. As technology advanced quickly, the need arose for specialized designers focused on optimizing the user's experience. The article explains that UX involves research, testing, understanding user emotions, flows, and behaviors when interacting with a product. UI involves the visual design colors, fonts, layouts, etc. UX drives UI choices by understanding user needs. The article argues UX and UI are interconnected you can't have good UI without the proper UX research behind it. The author believes professionals today likely already incorporate both UX and UI even if they don't realize it. She argues that separating UX and UI into isolated disciplines doesn't make sense, as they need to work together for an innovative product."
2245,1472750564,90383487344892230,en,tech,"Computer vision system inspects cookies using OpenCV and Raspberry Pi An educational project demonstrates using OpenCV and Raspberry Pi to analyze cookie images and detect defects in real time. The full code is available on GitHub. The author gained experience with microservices on a retail banking project. The project was successful in delivering features quickly using microservices, but the microservices were tightly coupled to legacy components. This tight coupling was due to several factors: Vertically scaled private data center infrastructure Lack of a cloud native development platform No self service database provisioning tool The microservices used the legacy system's large shared database. Access controls provided isolation between microservices. This allowed easy migration to separate databases later. However, it took 7 months to release the first microservice, due to risks of impacting production users. A framework defect caused connection issues during stress testing. The key lesson was that a microservice should encapsulate the unit of service and unit of failure from day one. This refers to a concept by Jim Gray modules should be units of service and failure, so failures don't propagate. When experts say microservices concepts aren't new, this quote by Jim Gray is a key historical reference."
2507,1475614359,-4996336942690402156,en,tech,"Three tips to improve productivity for developers An article recommends turning off notifications, avoiding unnecessary meetings, and working remotely to help developers avoid distractions and reach flow states where they do their best work. The article discusses how interruptions like meetings and notifications can destroy productivity and prevent developers from reaching flow states where they do their best work. Studies show open plan offices are disliked and reduce productivity, but employers use them to pack in more people. The article recommends three tips to improve productivity: 1. Turn off unnecessary notifications on your devices so you aren't constantly interrupted. Only leave on essential notifications. 2. Dodge unnecessary meetings by asking people to explain topics over email or chat instead of defaulting to meetings. Don't commit time unless it's clear a meeting is needed. 3. Ask for a private office or work from home if possible. Working remotely improves productivity by 13% partly by avoiding office distractions. If remote work isn't possible, a private office also helps avoid distractions. The article cites research on interruptions, open plan offices, working remotely, and achieving flow states. It argues we need to work asynchronously and protect focu

To change the user recommendations to account for recent changes in user behavior in real time we must first deploy an event tracker for the dataset group

In [153]:
event_tracker_name = 'news-event-tracker'

try: 
    create_event_tracker_response = personalize.create_event_tracker(
        name = event_tracker_name,
        datasetGroupArn=workshop_dataset_group_arn
        )
    event_tracker_arn = create_event_tracker_response['eventTrackerArn']
    print(json.dumps(create_event_tracker_response, indent=2))
    print ('\nCreating the event_tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    tracking_id = create_event_tracker_response['trackingId']
    print ('\nAnd trackingId = {}'.format(tracking_id))

except personalize.exceptions.ResourceAlreadyExistsException as e:
    event_tracker_list = personalize.list_event_trackers( 
        datasetGroupArn= workshop_dataset_group_arn
    )['eventTrackers']
    
    event_tracker_arn = event_tracker_list[0]['eventTrackerArn']
    
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    tracking_id = describe_event_tracker_response['eventTracker']['trackingId']
    
    print ('\nThe the Event Tracker with event_tracker_name {} already exists'.format(event_tracker_name))
    print ('\nWe will be using the existing Event Tracker with event_tracker_arn = {}'.format(event_tracker_arn))
    print ('\nAnd tracking_id = {}'.format(tracking_id))


The the Event Tracker with event_tracker_name news-event-tracker already exists

We will be using the existing Event Tracker with event_tracker_arn = arn:aws:personalize:us-east-1:381491864570:event-tracker/79f2d13c

And tracking_id = 4ea6be84-eb39-4b11-8f59-b76609ac56df


Now lets submit an interaction for our earlier user with one of our above items

In [154]:
response = personalize_events.put_events(
    trackingId=tracking_id,
    userId=sample_user,
    sessionId='session1',
    eventList=[
        {
            'eventId': 'madeupevent1',
            'eventType': 'VIEW',
            'itemId': '-1633984990770981161',
            'sentAt': 1714006143,
        },
    ]
)

In [155]:
response

{'ResponseMetadata': {'RequestId': '113349fd-8fec-46dc-b89f-87d003ff8670',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jul 2024 21:53:33 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '113349fd-8fec-46dc-b89f-87d003ff8670'},
  'RetryAttempts': 0}}

Now lets see how our recommendations change:

In [157]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5,
    filterArn = genre_filter_arn,
    filterValues = {"CUTOFF": "\"0\"", "GENRELIST": "\"tech\",\"non tech\",\"cloud provider news\",\"crypto currency\""},
    metadataColumns = {"ITEMS": ["training_text"]}
)

In [158]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [159]:
recommendations_df

,itemId,score,metadata
0,991271693336573226,0.031608,"{'training_text': 'PyLadies Teresina recreates viral meme to highlight sexism in IT A viral Facebook post from PyLadies Teresina used a popular meme to bring attention to the lack of women in IT jobs and the challenges they face entering the field. The WhatsApp meme \""Luiza, are you attentive?\"" was recreated by PyLadies Teresina to discuss sexism in IT. The meme shows a conversation between two friends where one explains a situation to the other. PyLadies Teresina reformulated the meme to highlight the lack of women in IT jobs. According to 2015 data, 92.1% of IT professionals are men, even though 74% of girls show interest in STEM in school. The meme aims to show how the IT field can be hostile to women. The Facebook post with the meme went viral. The post encourages women to pursue IT careers despite the maledominated environment.'}"
1,-6728844082024523434,0.019548,"{'training_text': 'Article discusses the soft skills that define senior engineers The article argues that technical expertise alone does not make a senior engineer. More important are soft skills like accountability, pragmatism, mentoring, and improving team culture. The article discusses what it means to be a senior engineer. It argues that knowing specific APIs is not what makes someone senior. Rather, senior engineers exhibit strong soft skills like reliability, accountability, flexibility, and pragmatism. They are team players who lead by example, listen to others, mentor junior engineers, and constantly look for ways to improve processes. The article emphasizes that senior engineers focus on delivering business value, not just writing clean code. They have the courage to speak up when they see unhealthy team dynamics. The key traits highlighted include: Being reliable and accountable Embracing change and new technologies Focusing on solving business problems Leading by example and mentoring others Listening to and learning from teammates Avoiding arrogance and leveraging authority Speaking up to improve team culture Driving process improvements The article concludes that technical skills naturally improve with experience, but soft skills require dedicated effort. Investing in soft skills can prevent career bottlenecks.'}"
2,-1415040208471067980,0.016108,"{'training_text': 'Google Cloud Platform releases icon library for diagrams Google Cloud Platform published downloadable icons and sample diagrams to help visually represent GCP infrastructure and services. Google Cloud Platform has released downloadable icon sets and sample diagrams to help developers, architects, and partners visually represent complex cloud infrastructure in documents and presentations. The icons cover GCP products, services, and elements that can be combined with other providers' icons to show hybrid/multicloud setups. Over 50 sample diagrams are available in Slide and PowerPoint formats to help kickstart new diagrams. As more products launch, Google will add icons so users should revisit the icon library. This allows accurate, uptodate visual representations of Google Cloud Platform architectures and services.'}"
3,8153556216916362816,0.015172,"{'training_text': 'Google announces Cloud Spanner, a globally distributed relational database Google Cloud Spanner combines the benefits of relational databases and NoSQL scalability. It aims to help applications scale without sacrificing data consistency. Google has announced Cloud Spanner, a new globally distributed relational database service. Cloud Spanner combines the benefits of traditional relational databases like ACID transactions and SQL with the scalability of NoSQL databases. This allows developers to build applications that can scale horizontally without sacrificing data consistency. Cloud Spanner is a fully managed service that provides automatic scaling, high availability, and disaster recovery. It has a simple pricing model based on compute usage, storage, and network egress

The top recommendations have changed quite a bit - though within the top five will still see a couple related to google technical products. Lets send in some more interactions, usually it is best practice to send in the interactoins as they occur so recommendations with the user can adjust in real time but you can submitted the interactions in batches of up to 10 and we will take advantage of this batching below. We will also say that these interactions are part of our most recent session.

In [160]:
response = personalize_events.put_events(
    trackingId=tracking_id,
    userId=sample_user,
    sessionId='session1',
    eventList=[
        {
            'eventId': 'madeupevent2',
            'eventType': 'VIEW',
            'itemId': '-1878128207048892154',
            'sentAt': 1714011143,
        },
        {
            'eventId': 'madeupevent3',
            'eventType': 'VIEW',
            'itemId': '-3058031327323357308',
            'sentAt': 1714013143,
        },
        {
            'eventId': 'madeupevent4',
            'eventType': 'VIEW',
            'itemId': '-4996336942690402156',
            'sentAt': 1714015143,
        },
        {
            'eventId': 'madeupevent5',
            'eventType': 'VIEW',
            'itemId': '90383487344892230',
            'sentAt': 1714018143,
        },
    ]
)

In [ ]:
Now lets see how this changes our recommendations

In [161]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5,
    filterArn = genre_filter_arn,
    filterValues = {"CUTOFF": "\"0\"", "GENRELIST": "\"tech\",\"non tech\",\"cloud provider news\",\"crypto currency\""},
    metadataColumns = {"ITEMS": ["training_text"]}
)

In [162]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])
recommendations_df

,itemId,score,metadata
0,-3307413259092352650,0.029518,"{'training_text': 'Ethereum platform allows decentralized blockchain applications with smart contracts Ethereum offers a decentralized blockchain platform for developers to build applications using smart contracts. It aims to improve on Bitcoin's limitations. Ethereum is a new blockchainbased software platform that allows developers to build decentralized applications. It was created by Vitalik Buterin and publicly released in 2015. Ethereum aims to improve on some of Bitcoin's limitations by offering more flexibility and allowing developers to create any kind of application on top of its blockchain. One key feature of Ethereum is its smart contract functionality, which allows agreements to be automatically executed when certain conditions are met. This enables complex applications to be built on top of Ethereum. The system is decentralized and run on a network of computers called nodes. Ethereum has generated significant interest from major banks and companies looking to utilize blockchain technology. However, it could face regulatory and technical challenges similar to those encountered by Bitcoin. While Bitcoin's creator remains anonymous, Ethereum was created in a more transparent manner by Buterin. Ethereum also has a dedicated team of developers building applications on the platform.'}"
1,2555983212310147009,0.023702,"{'training_text': 'Ethereum provides decentralized platform for blockchain apps and smart contracts The Ethereum blockchain system aims to enable decentralized applications and smart contracts. It has gained interest from major financial institutions for improving trading and transfers. Ethereum is a new blockchainbased software system that was recently publicly released. Ethereum aims to provide a decentralized platform for applications and smart contracts that use its native cryptocurrency, ether. The Ethereum system has garnered interest from major banks and financial institutions who are interested in using blockchain technology to improve efficiency and speed of trading and money transfers. Michael Novogratz, a former executive at Fortress Investment Group, has been investigating Ethereum since last fall. Unlike Bitcoin which was created anonymously, Ethereum was created in a more transparent manner by Vitalik Buterin, a 21yearold RussianCanadian programmer. The Ethereum network is run in a decentralized fashion by its users, and also has a dedicated team of developers building applications on the platform. According to cryptography expert Joseph Bonneau, Ethereum is the first blockchain system since Bitcoin to pique his interest. However, Ethereum could face some of the same technical challenges and legal issues regarding regulation that have affected Bitcoin.'}"
2,1649752043999819668,0.022001,"{'training_text': 'Node.js benefits and challenges for operations teams Node.js enables faster development but its singlethreaded, asynchronous nature causes issues like memory leaks and CPU blocking that require monitoring and diagnostics. Node.js is a fast growing platform used by many startups and enterprises. It acts as a new tier that connects legacy systems with new technologies. The benefits of Node.js include enabling faster development and deployment. However, its speed introduces challenges for operations and performance teams who focus on availability and performance. Some common Node.js problems these teams face include: 1. Memory leaks Node.js is prone to memory leaks which cause crashes. Heap dumps can help track down the cause. 2. CPU problems Node.js runs singlethreaded so CPUheavy operations can block requests. CPU sampling can identify where time is being spent. 3. Back pressure Slow backends can cause congestion in Node.js. Monitoring intertier communication can identify the root cause. 4. Security Node.js relies heavily on third party modules which can contain vulnerabilities. Scanning for issues and using a private module repository ca

There we go - now the all of the recommendations are different. The last thing we will go over is how to use a promotional filter. It is highly recommended to use a promotional filter on the publication date of an article to ensure that a certain percentage (perhaps even 100) of articles published are recent ones. Common filter periods include 2 weeks, 3 days, 1 day or even 12 hours depending on the particular asset we are looking to populate with articles.

In [174]:
get_recommendations_response = personalize_runtime.get_recommendations(
    promotions=[
        {
            'name': 'timefilter',
            'percentPromotedItems': 60,
            'filterArn': genre_filter_arn,
            'filterValues': {"CUTOFF": "\"1480000000\"", "GENRELIST": "\"tech\",\"non tech\",\"cloud provider news\",\"crypto currency\""}
        },
    ],
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5,
    filterArn = genre_filter_arn,
    filterValues = {"CUTOFF": "\"0\"", "GENRELIST": "\"tech\",\"non tech\",\"cloud provider news\",\"crypto currency\""},
    metadataColumns = {"ITEMS": ["training_text"]}
)

In [175]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])
recommendations_df

,itemId,score,promotionName,metadata
0,1873952032843878490,0.049597,timefilter,"{'training_text': 'Software teams adopt technologies based on hype instead of research, hurting projects The author argues teams should carefully evaluate new technologies instead of blindly adopting hypedriven ones that often don't fit their needs. Software development teams often adopt new technologies based on hype and trends rather than research, leading to poor architectural decisions. The author calls this \""Hype Driven Development\"" (HDD) and sees it as harmful, advocating instead for \""Solid Software Engineering\"". HDD has many forms: adopting a technology because of social media buzz, conferences, vocal team members, or copypasting Stack Overflow solutions without understanding. This results in bad decisions that hurt projects months or years later, sometimes requiring big rewrites. The hype cycle: A company solves a real problem with a new framework, talks about it, blurring complex details. Teams adopt it expecting benefits but get slowed down instead. Finally they realize the tradeoffs. Examples: React.js good for complex UIs but overused. \""TDD is dead\"" skipped tests create bugs. Microservices premature modularization hurts development. NoSQL poor fit and harder queries. Elixir/Phoenix learning curve slows teams. The author argues hype leads to hasty, poorly informed choices. New ideas spread quickly via social media, faster than their pros and cons are understood. Teams should carefully evaluate new technologies for their specific needs rather than assume they are a silver bullet.'}"
1,7065704533945771463,0.034244,timefilter,"{'training_text': 'Brazil's Central Bank may force Nubank to close down A new Central Bank rule in Brazil reducing credit card payment timelines could bankrupt fintech company Nubank, which lacks the capital reserves of large banks. Nubank, a major fintech company in Brazil, may have to close down if the Central Bank confirms a new rule on Tuesday, December 20 that would reduce the payment deadline from credit card companies to merchants from 30 days to just 2 days. This change would negatively impact all credit card companies, but especially smaller ones like Nubank that don't have the capital reserves of large banks. Nubank cofounder Cristina Junqueira criticized the idea, saying the current 30 day period allows time for the customer to pay their credit card bill, the issuer to receive the funds, and the purchaser to pay the retailer. She says reducing to 2 days would force Nubank to pay purchasers before even receiving customer payments. Simulations show that even 15 days would require Nubank to raise almost $1 billion in capital immediately to survive. Junqueira describes the potential change as \""apocalyptic\"" and says with just 2 days, Nubank would have to \""turn off the light and close the door.\"".'}"
2,6244532954645766056,0.031179,timefilter,"{'training_text': 'MIT Digital Currency Initiative explores blockchain applications MIT's Digital Currency Initiative is researching how blockchain could reshape the internet, improve privacy, and enable central bank digital currencies. The Massachusetts Institute of Technology (MIT) launched its Digital Currency Initiative (DCI) in 2015 to research blockchain technology. The initiative is led by former White House advisor Neha Narula and has 22 team members working on over 7 projects. DCI is researching how blockchain could reshape the internet and reduce 'echo chambers' where people only see news from limited sources. They are exploring IPFS and Blockstack which aim to decentralize control of information. However, Narula believes underlying economic incentives of megaplatforms like Facebook and Google are the core issue. Another focus is improving privacy and confidentiality in blockchains while retaining auditability. The 'Auditable Ledger' project explores this for banking ledgers, allowing auditors to verify systemlevel details while preserving user p

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in [`Retail_04_Clean_Up.ipynb`](Retail_04_Clean_Up.ipynb).